# Imports

In [1]:
import numpy as np, pandas as pd
import os, shutil
from distutils.dir_util import copy_tree

# Define source path for train and test data

In [2]:
'''
Define the path to the train and test data files on your local computer. 

src_path should be a directory which has two folders: 1) train, and 2) test
'train' folder must have a file called: ratings_train.csv
'test' folder must have a file called : ratings_test.csv 
'''

# path to your folder which has train and test folders in it 
src_path = f'./data/'

# Make Volume to Mount with appropriate folders
This is a folder created in this present directory. <br>
We will copy the data files into this folder and then bind mount this folder into the container. <br>
When the container is run, model artifacts and other outputs will also be saved to this same folder.<br>
These files on the shared volume will persist when the container is shut down.<br>

In [ ]:
# Run this as-is. Do not change any folder names!!! 
mounted_volume = 'ml_vol'
if os.path.exists(mounted_volume): shutil.rmtree(mounted_volume)

os.mkdir(mounted_volume)
subdirs = ['data', 'logs', 'model', 'output', 'score']
for subdir in subdirs: 
    sub_dir_path = os.path.join(mounted_volume, subdir)
    os.mkdir(sub_dir_path)

subdirs = ['train', 'test']
for subdir in subdirs: 
    sub_dir_path = os.path.join(mounted_volume, 'data', subdir)
    os.mkdir(sub_dir_path)

# Copy Data from Source Path Into Mounted Volume

In [4]:
dest_path = os.path.join(mounted_volume, 'data')
for input_type in ['train', 'test']:
    full_src = os.path.join(src_path, input_type)
    full_dest = os.path.join(dest_path, input_type)
    
    if os.path.exists(full_src): copy_tree(full_src, full_dest)

# Create Container From Image

In [ ]:
%%bash 
declare vol_path="$(pwd)/ml_vol"
docker run -d -p 3000:3000 -v $vol_path:/app/ml_vol --name mfc abudesai/rec_base_mf_res:1

# Check Container and Image

In [ ]:
%%bash 
docker ps

# Run Training¶

In [ ]:
%%bash 
docker exec mfc python train.py

# Run Test Data Predictions

In [ ]:
%%bash 
docker exec mfc python predict.py

# Score Test Data Predictions

In [ ]:
%%bash 
docker exec mfc python score.py

# Check Outputs In mounted volume
- data: this is where we mounted training and test data
- logs: contains logged model training output
- model: trained model artifacts
- output: contains predictions. Also contains output from hyper-parameter tuning, if run. 

In [ ]:
%%bash
ls ml_vol

# Stop Container and Remove Image

In [ ]:
%%bash
docker stop mfc

In [ ]:
%%bash
docker rmi abudesai/rec_base_mf_res:1